## Deterimining Optimal Trading Rules without Backtesting by Carr & Prado


Based on the paper by Peter P. Carr and Marcos Lopez de Prado on "Deterimining Optimal Trading Rules without Backtesting", this Python script simulates optimal trading strategies using the Ornstein-Uhlenbeck (O-U) process and aims to determine the optimal trading rules, specifically the profit-taking (PT) and stop-loss (SL) levels, without relying on historical backtesting. The O-U process is a mean-reverting stochastic process. 

Key Features:
- Fetching Real Stock Data: Utilizes yfinance to fetch historical stock price data.
- Parameter Estimation: Estimates parameters for the O-U process based on real stock data, including mean and standard deviation of log returns.
- Simulation of Price Movements: Simulates price movements using the O-U process, incorporating the estimated parameters.
- Optimization of Trading Rules: Evaluates various combinations of PT and SL thresholds to identify the pair that offers the best risk-adjusted return, as indicated by the Sharpe ratio.
- Progress Tracking: Includes progress bars for each simulation set using tqdm, enhancing the user experience especially for lengthy simulations.

Note:
1. The stop-loss (SL) level is a threshold for the absolute decrease in the simulated price from its initial value.
2. The profit-taking (PT) level is a threshold for the absolute increase in the simulated price from its initial value.


In [1]:
import yfinance as yf
import numpy as np
from random import gauss
from itertools import product
import pandas as pd
from tqdm import tqdm

# Fetch historical data from yfinance
def fetch_data(ticker, start_date, end_date):
    stock_data = yf.download(ticker, start=start_date, end=end_date)
    return stock_data['Close']

# Estimate parameters for the O-U process
def estimate_parameters(price_data):
    log_returns = np.log(price_data).diff().dropna()
    mu = np.mean(log_returns)
    sigma = np.std(log_returns)
    return mu, sigma

# Batch simulation for O-U process
def batch(coeffs, nIter=1e5, maxHP=100, rPT=np.linspace(0,10,21), rSLm=np.linspace(0,10,21)):
    phi = 2**(-1./coeffs['hl'])
    output1 = []
    for comb_ in product(rPT, rSLm):
        output2 = []
        for iter_ in tqdm(range(int(nIter)), desc=f"Processing {comb_}"):
            p, hp, seed = 0, 0, 0
            while True:
                p = (1-phi)*coeffs['forecast'] + phi*p + coeffs['sigma']*gauss(0,1)
                cP = p - seed; hp += 1
                if cP > comb_[0] or cP < -comb_[1] or hp > maxHP:
                    output2.append(cP)
                    break
        mean, std = np.mean(output2), np.std(output2)
        sharpe = mean/std if std != 0 else 0
        output1.append((comb_[0], comb_[1], mean, std, sharpe))
    return output1

# Main function
def main(ticker, start_date, end_date):
    price_data = fetch_data(ticker, start_date, end_date)
    mu, sigma = estimate_parameters(price_data)
    coeffs = {'forecast': mu, 'hl': 50, 'sigma': sigma}
    output = batch(coeffs, nIter=1e4, maxHP=100)
    return output

# Example usage
data = main('SPY', '1993-01-01', '2023-12-31')



[*********************100%***********************]  1 of 1 completed


Processing (1.0, 1.5): 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████| 10000/10000 [00:00<00:00, 13133.53it/s]


Processing (2.0, 3.5): 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████| 10000/10000 [00:00<00:00, 12859.26it/s]


Processing (3.0, 5.5): 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████| 10000/10000 [00:00<00:00, 13496.76it/s]


Processing (4.0, 7.5): 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████| 10000/10000 [00:00<00:00, 13499.68it/s]


...

Processing (6.5, 1.0): 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████| 10000/10000 [00:00<00:00, 13404.97it/s]


Processing (7.5, 3.0): 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████| 10000/10000 [00:00<00:00, 13454.24it/s]


Processing (8.5, 5.0): 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████| 10000/10000 [00:00<00:00, 13288.33it/s]


Processing (9.5, 7.0): 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████| 10000/10000 [00:00<00:00, 13301.69it/s]


Processing (10.0, 10.0): 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 10000/10000 [00:00<00:00, 13341.15it/s]


In [2]:
# Process the data into a DataFrame
df = pd.DataFrame(data, columns=['PT', 'SL', 'Mean_Return', 'Std_Dev', 'Sharpe_Ratio'])
df = df.astype({'PT': float, 'SL': float, 'Mean_Return': float, 'Std_Dev': float, 'Sharpe_Ratio': float})

# Analyze to find the optimal trading rules
optimal_rule = df.loc[df['Sharpe_Ratio'].idxmax()]

print(f"Optimal Trading Rule based on highest Sharpe Ratio:\n{optimal_rule}")

Optimal Trading Rule based on highest Sharpe Ratio:
PT              0.000000
SL              8.500000
Mean_Return     0.006415
Std_Dev         0.016663
Sharpe_Ratio    0.384986
Name: 17, dtype: float64
